In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE=256
BATCH_SIZE=32
dataset=tf.keras.preprocessing.image_dataset_from_directory(
"PlantVillage",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
batch_size=BATCH_SIZE)

In [ ]:
class_names=dataset.class_names

In [ ]:
class_names

In [ ]:
len(dataset)

In [ ]:
for image_batch, label_batch in dataset.take(1):
    normalized_image = (image_batch[0].numpy() / 255.0).astype("float32")
    plt.imshow(normalized_image, interpolation='nearest')
    plt.show()


In [ ]:
for image_batch, label_batch in dataset.take(1):
    plt.imshow(image_batch[0].numpy().astype("uint8"), interpolation='nearest')
    plt.show()

In [ ]:
print(image_batch[0].shape)


In [ ]:
for image_batch, label_batch in dataset.take(1):
    for i in range(15):
        ax=plt.subplot(3,5,i+1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[label_batch[i]])
    

In [ ]:
train_size=0.8
len(dataset)*train_size

In [ ]:
train_ds=dataset.take(516)
len(train_ds)

In [ ]:
test_ds=dataset.skip(516)
len(test_ds)

In [ ]:
val_size=0.1
len(dataset)*val_size

In [ ]:
val_ds=test_ds.take(64)
len(val_ds)

In [ ]:
test_ds=test_ds.skip(64)
len(test_ds)

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, shuffle_size=10000):
    ds_size = len(ds)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)  # Fix the parameter name here
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)


In [ ]:
len(train_ds)

In [ ]:
len(val_ds)

In [ ]:
len(test_ds)

In [ ]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)


In [ ]:
resize_and_rescale=tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)])

In [ ]:
data_augmentation =tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
])
train_ds = train_ds.map(lambda x,y:(data_augmentation(x, training = True),y)).prefetch(buffer_size=tf.data.AUTOTUNE)


In [ ]:
from tensorflow.keras import models, layers

CHANNELS = 3

EPOCHS = 50
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 15

# Assuming you have a resize_and_rescale layer defined
resize_and_rescale = layers.experimental.preprocessing.Rescaling(1./255)

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),  # Add dropout with a rate of 0.25
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),  # Add dropout before the output layer with a rate of 0.5
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)


In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=50,
)

In [ ]:
scores = model.evaluate(test_ds)


In [ ]:
scores

In [ ]:
history


In [ ]:
history.history.keys()

In [ ]:
history.params

In [ ]:
history.history['accuracy']

In [ ]:
acc=history.history['accuracy']
val_acc=history.history["val_accuracy"]
loss=history.history["loss"]
val_loss=history.history["val_loss"]

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS),acc,label="Training accuracy")
plt.plot(range(EPOCHS),val_acc,label="Validation accuracy")
plt.legend(loc='lower right')
plt.title('Training and Validation accuracy')


In [ ]:
plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import numpy as np
for images_batch, labels_batch in test_ds.take(1):
    
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])
    
    batch_prediction = model.predict(images_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(15):
        ax = plt.subplot(5, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")

In [ ]:
import os

# List all items in the directory
directory_contents = os.listdir("../taiwan")

# Filter out non-integer items
numeric_contents = [i for i in directory_contents if i.isdigit()]

# Convert to integers and add 0 to the list if no numeric items are found
numeric_values = [int(i) for i in numeric_contents] + [0]

# Find the maximum and increment
model_version = max(numeric_values) + 1

# Save the model with the new version
model.save(f"../taiwan/models/{model_version}")


In [ ]:
# Check if the file exists
if os.path.exists( r"../taiwan/PlantVillage/Pepper__bell___Bacterial_spot/0022d6b7-d47c-4ee2-ae9a-392a53f48647___JR_B.Spot%208964.JPG"):
    print("File exists.")
else:
    print("File does not exist.")

In [ ]:
from keras.models import load_model
import tensorflow as tf

# Define the latest model version
latest_model_version = 1  # Replace with the actual version number

# Load the model
loaded_model = load_model(f"../taiwan/models/{latest_model_version}")

# Convert the model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

# Save the TFLite model with standard Python file handling
model_path = 'C:/Users/lamas/Desktop/Ml-project/taiwan/model.tflite'
try:
    with open(model_path, 'wb') as f:
        f.write(tflite_model)
    print("Model converted and saved successfully.")
except Exception as e:
    print(f"Error: {e}")


In [ ]:
import tensorflow as tf

# Load the TFLite model
model_path = 'C:/Users/lamas/Desktop/Ml-project/taiwan/model.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get details about the input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:")
print(input_details)

print("\nOutput details:")
print(output_details)


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Load the TFLite model
model_path = 'C:/Users/lamas/Desktop/Ml-project/taiwan/model.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and preprocess an image
image_path = r'C:\Users\lamas\Downloads\le.jpeg'

img = tf.keras.preprocessing.image.load_img(image_path, target_size=(256, 256))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
input_data = img_array.astype(np.float32) / 255.0  # Normalize to the range [0, 1]

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Perform inference
interpreter.invoke()

# Get output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Process or analyze the output_data as needed for your application
print("Output data:", output_data)
predicted_class_index = np.argmax(output_data)
print("Predicted class index:", predicted_class_index)

# Visualization (replace this with your actual prediction logic)
plt.imshow(img)
plt.title(f"Predicted class index: {predicted_class_index}")
plt.show()
